In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.impute import KNNImputer

import warnings
warnings.filterwarnings(action='ignore')

In [43]:
weather_2020 = pd.read_csv('2020_weather.csv', sep=',', encoding='cp949')
weather_2021 = pd.read_csv('2021_weather.csv', sep=',', encoding='cp949')

In [2]:
weather_2022 = pd.read_csv('weather_test.csv', sep=',', encoding='cp949')

In [97]:
weather_final = pd.read_csv('final_data.csv', sep=',', encoding='cp949')

In [2]:
weather_2020

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),시정(10m),지면온도(°C)
0,108,서울,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38,1.4,-18.5,1022.6,1033.8,NaN,NaN,NaN,2000.0,-2.8
1,108,서울,2020-01-01 01:00,-5.9,NaN,1.7,50.0,40,1.6,-17.3,1021.8,1033.0,NaN,NaN,NaN,2000.0,-2.4
2,108,서울,2020-01-01 02:00,-5.7,NaN,0.1,0.0,42,1.7,-16.5,1021.6,1032.8,NaN,NaN,NaN,1988.0,-2.4
3,108,서울,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46,1.8,-15.4,1021.5,1032.7,NaN,NaN,NaN,2000.0,-2.7
4,108,서울,2020-01-01 04:00,-5.4,NaN,0.0,0.0,50,2.0,-14.2,1021.6,1032.8,NaN,NaN,NaN,1908.0,-2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17563,159,부산,2020-12-31 19:00,0.6,NaN,6.2,270.0,41,2.6,-11.1,1014.0,1022.8,NaN,NaN,NaN,5000.0,-0.5
17564,159,부산,2020-12-31 20:00,0.1,NaN,6.0,270.0,44,2.7,-10.7,1014.8,1023.6,NaN,NaN,NaN,5000.0,-0.8
17565,159,부산,2020-12-31 21:00,-0.2,NaN,5.0,290.0,48,2.9,-9.9,1014.6,1023.4,NaN,NaN,NaN,5000.0,-1.0
17566,159,부산,2020-12-31 22:00,-0.7,NaN,2.7,270.0,51,3.0,-9.6,1014.8,1023.6,NaN,NaN,NaN,5000.0,-1.3


In [3]:
weather_2020.columns

Index(['지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)',
       '일사(MJ/m2)', '적설(cm)', '시정(10m)', '지면온도(°C)'],
      dtype='object')

In [98]:
busan_cols = []
seoul_cols = []

for i in weather_final.columns:
    busan_cols.append('부산 '+i)
    seoul_cols.append('서울 '+i)
busan_cols, seoul_cols

(['부산 지점',
  '부산 지점명',
  '부산 일시',
  '부산 기온(°C)',
  '부산 강수량(mm)',
  '부산 풍속(m/s)',
  '부산 풍향(16방위)',
  '부산 습도(%)',
  '부산 증기압(hPa)',
  '부산 이슬점온도(°C)',
  '부산 현지기압(hPa)',
  '부산 해면기압(hPa)',
  '부산 일조(hr)',
  '부산 일사(MJ/m2)',
  '부산 적설(cm)',
  '부산 시정(10m)',
  '부산 지면온도(°C)'],
 ['서울 지점',
  '서울 지점명',
  '서울 일시',
  '서울 기온(°C)',
  '서울 강수량(mm)',
  '서울 풍속(m/s)',
  '서울 풍향(16방위)',
  '서울 습도(%)',
  '서울 증기압(hPa)',
  '서울 이슬점온도(°C)',
  '서울 현지기압(hPa)',
  '서울 해면기압(hPa)',
  '서울 일조(hr)',
  '서울 일사(MJ/m2)',
  '서울 적설(cm)',
  '서울 시정(10m)',
  '서울 지면온도(°C)'])

1. Weather train data

In [45]:
weather_data = pd.DataFrame()

for data in [weather_2020, weather_2021]:
    busan_weather = data.loc[data['지점']==159]
    busan_weather.reset_index(inplace=True, drop=True)
    busan_weather.columns = busan_cols
    
    seoul_weather = data.loc[data['지점']==108]
    seoul_weather.columns = seoul_cols
    
    new_data = pd.DataFrame(seoul_weather.loc[:, '서울 일시'])
    new_data.columns = ['일시']

    weather = pd.concat([seoul_weather.iloc[:, 3:], busan_weather.iloc[:, 3:]], axis=1)
    new_data = pd.concat([new_data, weather], axis=1)
    weather_data = pd.concat([weather_data, new_data], axis=0)

In [46]:
weather_data.to_csv('weather_data.csv', index=False)

In [29]:
weather_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),...,부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C)
0,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38,1.4,-18.5,1022.6,1033.8,...,23,1.2,-20.8,1021.7,1030.7,NaN,NaN,NaN,2000.0,-1.4
1,2020-01-01 01:00,-5.9,NaN,1.7,50.0,40,1.6,-17.3,1021.8,1033.0,...,22,1.1,-21.7,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.6
2,2020-01-01 02:00,-5.7,NaN,0.1,0.0,42,1.7,-16.5,1021.6,1032.8,...,23,1.1,-21.3,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.9
3,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46,1.8,-15.4,1021.5,1032.7,...,24,1.2,-20.6,1021.1,1030.1,NaN,NaN,NaN,2000.0,-2.0
4,2020-01-01 04:00,-5.4,NaN,0.0,0.0,50,2.0,-14.2,1021.6,1032.8,...,35,1.7,-16.4,1021.0,1030.0,NaN,NaN,NaN,2000.0,-2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31 19:00,-6.4,NaN,2.2,250.0,34,1.3,-19.7,1021.1,1032.3,...,22,1.4,-18.8,1019.1,1028.0,NaN,NaN,NaN,5000.0,-1.6
8756,2021-12-31 20:00,-6.3,NaN,4.1,320.0,35,1.3,-19.2,1021.2,1032.4,...,22,1.3,-19.2,1019.1,1028.0,NaN,NaN,NaN,5000.0,-2.1
8757,2021-12-31 21:00,-6.7,NaN,4.8,320.0,36,1.3,-19.3,1021.2,1032.4,...,23,1.4,-19.0,1018.7,1027.6,NaN,NaN,NaN,5000.0,-2.5
8758,2021-12-31 22:00,-7.5,NaN,3.0,320.0,37,1.3,-19.7,1021.8,1033.1,...,23,1.3,-19.4,1019.5,1028.4,NaN,NaN,NaN,5000.0,-2.2


---
2. Weather test data

In [5]:
weather_test_data = pd.DataFrame()

busan_weather = weather_2022.loc[weather_2022['지점']==159]
busan_weather.reset_index(inplace=True, drop=True)
busan_weather.columns = busan_cols

seoul_weather = weather_2022.loc[weather_2022['지점']==108]
seoul_weather.columns = seoul_cols

new_data = pd.DataFrame(seoul_weather.loc[:, '서울 일시'])
new_data.columns = ['일시']

weather = pd.concat([seoul_weather.iloc[:, 3:], busan_weather.iloc[:, 3:]], axis=1)
new_data = pd.concat([new_data, weather], axis=1)
weather_test_data = pd.concat([weather_test_data, new_data], axis=0)

In [6]:
weather_test_data.to_csv('weather_test_data.csv', index=False)

3. Weather final data

In [99]:
weather_final_data = pd.DataFrame()

busan_weather = weather_final.loc[weather_final['지점']==159]
busan_weather.reset_index(inplace=True, drop=True)
busan_weather.columns = busan_cols

seoul_weather = weather_final.loc[weather_final['지점']==108]
seoul_weather.columns = seoul_cols

new_data = pd.DataFrame(seoul_weather.loc[:, '서울 일시'])
new_data.columns = ['일시']

weather = pd.concat([seoul_weather.iloc[:, 3:], busan_weather.iloc[:, 3:]], axis=1)
new_data = pd.concat([new_data, weather], axis=1)
weather_final_data = pd.concat([weather_final_data, new_data], axis=0)

In [100]:
weather_final_data.to_csv('weather_final_data.csv', index=False)

---

3. time train data

In [49]:
time_data = pd.DataFrame()

for year in ['2020', '2021']:
    for month in range(1, 13):
        data = pd.read_csv(year+'_'+str(month)+'월.csv', sep=',', encoding='cp949')
        for i in range(len(data.columns)-1, 3, -1):
            new_data = data.iloc[:, i]
            time_data = pd.concat([time_data, new_data], axis=0)
time_data.columns = ['소요시간']

In [50]:
time_data

,소요시간
0,0
1,0
2,04:51
3,04:44
4,06:03
...,...
19,05:09
20,05:20
21,05:23
22,05:55


4. time test data

In [7]:
time_test_data = pd.DataFrame()

for month in range(1, 10):
    data = pd.read_csv('2022_'+str(month)+'월.csv', sep=',', encoding='cp949')
    for i in range(len(data.columns)-1, 3, -1):
        new_data = data.iloc[:, i]
        time_test_data = pd.concat([time_test_data, new_data], axis=0)
time_test_data.columns = ['소요시간']

In [8]:
time_test_data

,소요시간
0,03:47
1,03:56
2,0
3,03:53
4,03:49
...,...
18,04:49
19,04:39
20,04:40
21,04:34


In [101]:
weather_data.reset_index(inplace=True, drop=True)
weather_test_data.reset_index(inplace=True, drop=True)
weather_final_data.reset_index(inplace=True, drop=True)
time_data.reset_index(inplace=True, drop=True)
time_test_data.reset_index(inplace=True, drop=True)

In [54]:
train_data = weather_data.copy()
train_data = pd.concat([train_data, time_data], axis=1)
pd.set_option('display.max_columns', train_data.shape[0])
train_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간
0,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38,1.4,-18.5,1022.6,1033.8,NaN,NaN,NaN,2000.0,-2.8,-2.5,NaN,1.7,360.0,23,1.2,-20.8,1021.7,1030.7,NaN,NaN,NaN,2000.0,-1.4,0
1,2020-01-01 01:00,-5.9,NaN,1.7,50.0,40,1.6,-17.3,1021.8,1033.0,NaN,NaN,NaN,2000.0,-2.4,-2.9,NaN,2.0,340.0,22,1.1,-21.7,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.6,0
2,2020-01-01 02:00,-5.7,NaN,0.1,0.0,42,1.7,-16.5,1021.6,1032.8,NaN,NaN,NaN,1988.0,-2.4,-3.1,NaN,1.9,140.0,23,1.1,-21.3,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.9,04:51
3,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46,1.8,-15.4,1021.5,1032.7,NaN,NaN,NaN,2000.0,-2.7,-2.8,NaN,4.5,20.0,24,1.2,-20.6,1021.1,1030.1,NaN,NaN,NaN,2000.0,-2.0,04:44
4,2020-01-01 04:00,-5.4,NaN,0.0,0.0,50,2.0,-14.2,1021.6,1032.8,NaN,NaN,NaN,1908.0,-2.5,-3.1,NaN,1.9,110.0,35,1.7,-16.4,1021.0,1030.0,NaN,NaN,NaN,2000.0,-2.1,06:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2021-12-31 19:00,-6.4,NaN,2.2,250.0,34,1.3,-19.7,1021.1,1032.3,NaN,NaN,NaN,2000.0,-4.5,0.5,NaN,2.0,320.0,22,1.4,-18.8,1019.1,1028.0,NaN,NaN,NaN,5000.0,-1.6,05:09
17540,2021-12-31 20:00,-6.3,NaN,4.1,320.0,35,1.3,-19.2,1021.2,1032.4,NaN,NaN,NaN,2000.0,-5.3,0.0,NaN,2.2,320.0,22,1.3,-19.2,1019.1,1028.0,NaN,NaN,NaN,5000.0,-2.1,05:20
17541,2021-12-31 21:00,-6.7,NaN,4.8,320.0,36,1.3,-19.3,1021.2,1032.4,NaN,NaN,NaN,2000.0,-5.7,-0.4,NaN,4.1,290.0,23,1.4,-19.0,1018.7,1027.6,NaN,NaN,NaN,5000.0,-2.5,05:23
17542,2021-12-31 22:00,-7.5,NaN,3.0,320.0,37,1.3,-19.7,1021.8,1033.1,NaN,NaN,NaN,2000.0,-6.2,-0.8,NaN,3.2,250.0,23,1.3,-19.4,1019.5,1028.4,NaN,NaN,NaN,5000.0,-2.2,05:55


In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   일시            17544 non-null  object 
 1   서울 기온(°C)     17543 non-null  float64
 2   서울 강수량(mm)    2010 non-null   float64
 3   서울 풍속(m/s)    17542 non-null  float64
 4   서울 풍향(16방위)   17542 non-null  float64
 5   서울 습도(%)      17544 non-null  int64  
 6   서울 증기압(hPa)   17542 non-null  float64
 7   서울 이슬점온도(°C)  17537 non-null  float64
 8   서울 현지기압(hPa)  17531 non-null  float64
 9   서울 해면기압(hPa)  17531 non-null  float64
 10  서울 일조(hr)     9593 non-null   float64
 11  서울 일사(MJ/m2)  9593 non-null   float64
 12  서울 적설(cm)     587 non-null    float64
 13  서울 시정(10m)    17544 non-null  float64
 14  서울 지면온도(°C)   17531 non-null  float64
 15  부산 기온(°C)     17539 non-null  float64
 16  부산 강수량(mm)    1820 non-null   float64
 17  부산 풍속(m/s)    17517 non-null  float64
 18  부산 풍향(16방위)   17517 non-nu

In [11]:
test_data = weather_test_data.copy()
test_data = pd.concat([test_data, time_test_data], axis=1)
pd.set_option('display.max_columns', test_data.shape[0])
test_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간
0,2022-01-01 00:00,-8.5,NaN,1.9,320,41,1.3,-19.4,1021.9,1033.2,NaN,NaN,NaN,2000.0,-7.0,-1.6,NaN,7.2,290,25,1.4,-19.1,1019.7,1028.6,NaN,NaN,NaN,5000.0,-4.4,03:47
1,2022-01-01 01:00,-9.2,NaN,1.8,270,42,1.3,-19.7,1022.2,1033.6,NaN,NaN,NaN,2000.0,-7.2,-2.0,NaN,3.6,200,24,1.3,-19.9,1019.9,1028.8,NaN,NaN,NaN,5000.0,-4.7,03:56
2,2022-01-01 02:00,-9.5,NaN,1.2,270,43,1.3,-19.7,1022.3,1033.7,NaN,NaN,NaN,2000.0,-7.5,-2.3,NaN,2.4,230,24,1.2,-20.1,1020.3,1029.3,NaN,NaN,NaN,5000.0,-4.7,0
3,2022-01-01 03:00,-9.3,NaN,1.4,290,46,1.4,-18.8,1022.6,1034.0,NaN,NaN,NaN,2000.0,-7.6,-2.8,NaN,3.5,270,27,1.3,-19.2,1020.6,1029.6,NaN,NaN,NaN,5000.0,-5.3,03:53
4,2022-01-01 04:00,-9.6,NaN,1.7,340,48,1.4,-18.5,1021.9,1033.3,NaN,NaN,NaN,2000.0,-7.6,-2.3,NaN,2.4,230,20,1.0,-22.2,1020.0,1029.0,NaN,NaN,NaN,5000.0,-5.5,03:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971,2022-09-06 19:00,23.2,NaN,3.2,250,66,18.6,16.4,999.6,1009.4,0.5,0.17,NaN,2000.0,20.1,24.5,NaN,1.8,320,64,19.6,17.2,1000.8,1008.7,0.4,0.11,NaN,5000.0,25.1,NaN
5972,2022-09-06 20:00,22.1,NaN,1.8,250,68,18.1,15.9,1000.6,1010.4,NaN,NaN,NaN,2000.0,18.7,23.5,NaN,1.3,320,67,19.4,17.0,1001.7,1009.6,NaN,NaN,NaN,5000.0,24.2,NaN
5973,2022-09-06 21:00,21.2,NaN,1.8,250,71,17.8,15.7,1001.8,1011.7,NaN,NaN,NaN,2000.0,17.8,23.0,NaN,1.0,290,69,19.2,16.9,1002.5,1010.4,NaN,NaN,NaN,5000.0,23.5,NaN
5974,2022-09-06 22:00,20.6,NaN,1.4,270,74,17.8,15.7,1002.7,1012.6,NaN,NaN,NaN,2000.0,17.5,22.1,NaN,1.2,320,72,19.1,16.8,1003.3,1011.3,NaN,NaN,NaN,5000.0,22.8,NaN


In [102]:
final_data = weather_final_data.copy()
pd.set_option('display.max_columns', weather_final_data.shape[1])
final_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C)
0,2022-09-07 0:00,20.7,NaN,3.4,250,83,20.2,17.7,1003.1,1013.0,NaN,NaN,NaN,2000,17.7,21.4,NaN,2.2,290,74,18.8,16.5,1004.3,1012.3,NaN,NaN,NaN,5000,22.0
1,2022-09-07 1:00,20.7,NaN,2.2,250,86,20.9,18.2,1003.3,1013.2,NaN,NaN,NaN,2000,18.1,20.9,NaN,2.4,290,77,19.0,16.7,1004.5,1012.6,NaN,NaN,NaN,5000,21.7
2,2022-09-07 2:00,20.7,NaN,2.5,230,89,21.7,18.8,1003.3,1013.2,NaN,NaN,NaN,1871,18.2,20.5,NaN,2.9,290,76,18.3,16.1,1004.6,1012.7,NaN,NaN,NaN,5000,21.2
3,2022-09-07 3:00,20.5,NaN,1.9,230,91,21.8,18.9,1003.8,1013.8,NaN,NaN,NaN,1155,17.5,20.3,NaN,2.9,320,73,17.3,15.2,1005.2,1013.3,NaN,NaN,NaN,5000,20.9
4,2022-09-07 4:00,19.9,NaN,0.7,270,92,21.3,18.5,1004.0,1014.0,NaN,NaN,NaN,881,16.6,20.0,NaN,4.1,290,73,17.0,15.0,1005.3,1013.4,NaN,NaN,NaN,5000,20.5
5,2022-09-07 5:00,19.1,NaN,1.3,230,92,20.2,17.7,1004.6,1014.5,NaN,NaN,NaN,711,15.7,19.5,NaN,3.3,320,71,16.0,14.0,1005.6,1013.7,NaN,NaN,NaN,5000,20.2
6,2022-09-07 6:00,18.6,NaN,1.5,290,94,20.1,17.6,1005.3,1015.3,NaN,NaN,NaN,887,15.6,18.9,NaN,3.9,290,75,16.3,14.3,1006.4,1014.5,NaN,NaN,NaN,5000,20.0
7,2022-09-07 7:00,18.0,NaN,0.7,50,93,19.1,16.8,1006.2,1016.2,0.4,0.12,NaN,1992,15.6,19.6,NaN,1.3,270,63,14.3,12.3,1007.2,1015.3,0.0,0.08,NaN,5000,20.4
8,2022-09-07 8:00,19.3,NaN,1.2,20,89,19.9,17.4,1006.8,1016.8,1.0,0.74,NaN,2000,21.0,21.7,NaN,1.2,180,53,13.7,11.6,1007.6,1015.6,0.9,0.84,NaN,5000,22.6
9,2022-09-07 9:00,21.6,NaN,2.0,20,81,20.8,18.1,1007.2,1017.1,1.0,1.53,NaN,2000,27.6,23.9,NaN,1.2,230,47,13.8,11.8,1007.9,1015.9,1.0,1.63,NaN,5000,25.6


In [103]:
train_data['서울 풍향(16방위)'] = np.radians(train_data['서울 풍향(16방위)'])
train_data['부산 풍향(16방위)'] = np.radians(train_data['부산 풍향(16방위)'])
test_data['서울 풍향(16방위)'] = np.radians(test_data['서울 풍향(16방위)'])
test_data['부산 풍향(16방위)'] = np.radians(test_data['부산 풍향(16방위)'])
final_data['서울 풍향(16방위)'] = np.radians(final_data['서울 풍향(16방위)'])
final_data['부산 풍향(16방위)'] = np.radians(final_data['부산 풍향(16방위)'])

In [60]:
train_data.isna().sum()

일시                  0
서울 기온(°C)           1
서울 강수량(mm)      15534
서울 풍속(m/s)          2
서울 풍향(16방위)         2
서울 습도(%)            0
서울 증기압(hPa)         2
서울 이슬점온도(°C)        7
서울 현지기압(hPa)       13
서울 해면기압(hPa)       13
서울 일조(hr)        7951
서울 일사(MJ/m2)     7951
서울 적설(cm)       16957
서울 시정(10m)          0
서울 지면온도(°C)        13
부산 기온(°C)           5
부산 강수량(mm)      15724
부산 풍속(m/s)         27
부산 풍향(16방위)        27
부산 습도(%)            0
부산 증기압(hPa)         4
부산 이슬점온도(°C)        6
부산 현지기압(hPa)        4
부산 해면기압(hPa)        1
부산 일조(hr)        7955
부산 일사(MJ/m2)     7962
부산 적설(cm)       17544
부산 시정(10m)          8
부산 지면온도(°C)         0
소요시간                0
dtype: int64

In [46]:
test_data.columns

Index(['일시', '서울 기온(°C)', '서울 강수량(mm)', '서울 풍속(m/s)', '서울 풍향(16방위)',
       '서울 습도(%)', '서울 증기압(hPa)', '서울 이슬점온도(°C)', '서울 현지기압(hPa)',
       '서울 해면기압(hPa)', '서울 일조(hr)', '서울 일사(MJ/m2)', '서울 적설(cm)', '서울 시정(10m)',
       '서울 지면온도(°C)', '부산 기온(°C)', '부산 강수량(mm)', '부산 풍속(m/s)', '부산 풍향(16방위)',
       '부산 습도(%)', '부산 증기압(hPa)', '부산 이슬점온도(°C)', '부산 현지기압(hPa)',
       '부산 해면기압(hPa)', '부산 일조(hr)', '부산 일사(MJ/m2)', '부산 적설(cm)', '부산 시정(10m)',
       '부산 지면온도(°C)', '소요시간', 'holiday', 'Korean_holiday', 'year', 'month',
       'time', 'weekend'],
      dtype='object')

In [104]:
cols = ['서울 강수량(mm)', '서울 일조(hr)', '서울 일사(MJ/m2)', '서울 적설(cm)', '부산 강수량(mm)', '부산 일조(hr)', '부산 일사(MJ/m2)', '부산 적설(cm)']
for col in cols:
    train_data[col].fillna(0, inplace=True)
    test_data[col].fillna(0, inplace=True)
    final_data[col].fillna(0, inplace=True)

In [48]:
test_data.isna().sum()

일시                0
서울 기온(°C)         0
서울 강수량(mm)        0
서울 풍속(m/s)        0
서울 풍향(16방위)       0
서울 습도(%)          0
서울 증기압(hPa)       0
서울 이슬점온도(°C)      0
서울 현지기압(hPa)      0
서울 해면기압(hPa)      0
서울 일조(hr)         0
서울 일사(MJ/m2)      0
서울 적설(cm)         0
서울 시정(10m)        0
서울 지면온도(°C)       0
부산 기온(°C)         0
부산 강수량(mm)        0
부산 풍속(m/s)        0
부산 풍향(16방위)       0
부산 습도(%)          0
부산 증기압(hPa)       0
부산 이슬점온도(°C)      0
부산 현지기압(hPa)      0
부산 해면기압(hPa)      0
부산 일조(hr)         0
부산 일사(MJ/m2)      0
부산 적설(cm)         0
부산 시정(10m)        0
부산 지면온도(°C)       0
소요시간              0
holiday           0
Korean_holiday    0
year              0
month             0
time              0
weekend           0
dtype: int64

In [105]:
from sklearn.preprocessing import LabelEncoder

cols = ['서울 풍향(16방위)','부산 풍향(16방위)', '서울 일조(hr)', '부산 일조(hr)']
for feature in cols:
    # LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행. 
    encoder = LabelEncoder()
    encoder.fit(train_data[feature])
    train_data[feature] = encoder.transform(train_data[feature])
    
    encoder.fit(test_data[feature])
    test_data[feature] = encoder.transform(test_data[feature])
    
    encoder.fit(final_data[feature])
    final_data[feature] = encoder.transform(final_data[feature])

In [50]:
train_data['소요시간'] = train_data['소요시간'].replace('0', np.NaN)
test_data['소요시간'] = test_data['소요시간'].replace('0', np.NaN)

In [51]:
for index, row in train_data.iterrows():
    tmp = row['소요시간']
    if ':' not in str(tmp):
        continue
    h, m = str(tmp).split(':')
    row['소요시간'] = int(h)*60+int(m)
    train_data.iloc[index] = row

In [52]:
train_data['소요시간'] = train_data['소요시간'].astype(float)

In [53]:
for index, row in test_data.iterrows():
    tmp = row['소요시간']
    if ':' not in str(tmp):
        continue
    h, m = str(tmp).split(':')
    row['소요시간'] = int(h)*60+int(m)
    test_data.iloc[index] = row

In [54]:
test_data['소요시간'] = test_data['소요시간'].astype(float)

In [106]:
final_data['소요시간'] = 0

In [107]:
tmp_train = train_data.iloc[:, 1:]
tmp_test = test_data.iloc[:, 1:]
tmp_final = final_data.iloc[:, 1:]

In [110]:
imputer = KNNImputer()
tmp_train = pd.DataFrame(imputer.fit_transform(tmp_train), columns=tmp_train.columns)
tmp_test = pd.DataFrame(imputer.fit_transform(tmp_test), columns=tmp_test.columns)
tmp_final = pd.DataFrame(imputer.fit_transform(tmp_final), columns=tmp_final.columns)

In [114]:
for feature in tmp_final.columns[1:]:
    train_data[feature] = tmp_train[feature]
    test_data[feature] = tmp_test[feature]
    final_data[feature] = tmp_final[feature]

In [24]:
train_data['소요시간'] = train_data['소요시간'].astype(int)
test_data['소요시간'] = test_data['소요시간'].astype(int)

In [25]:
test_data.isna().sum()

일시              0
서울 기온(°C)       0
서울 강수량(mm)      0
서울 풍속(m/s)      0
서울 풍향(16방위)     0
서울 습도(%)        0
서울 증기압(hPa)     0
서울 이슬점온도(°C)    0
서울 현지기압(hPa)    0
서울 해면기압(hPa)    0
서울 일조(hr)       0
서울 일사(MJ/m2)    0
서울 적설(cm)       0
서울 시정(10m)      0
서울 지면온도(°C)     0
부산 기온(°C)       0
부산 강수량(mm)      0
부산 풍속(m/s)      0
부산 풍향(16방위)     0
부산 습도(%)        0
부산 증기압(hPa)     0
부산 이슬점온도(°C)    0
부산 현지기압(hPa)    0
부산 해면기압(hPa)    0
부산 일조(hr)       0
부산 일사(MJ/m2)    0
부산 적설(cm)       0
부산 시정(10m)      0
부산 지면온도(°C)     0
소요시간            0
dtype: int64

In [26]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

### 공휴일, 요일, 명절여부 추가하기

1. 일시 -> Datetime

In [81]:
train_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간
0,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38.0,1.4,-18.5,1022.6,1033.8,0.0,0.0,0.0,2000.0,-2.8,-2.5,0.0,1.7,16.0,23.0,1.2,-20.8,1021.7,1030.7,0.0,0.0,0.0,2000.0,-1.4,265
1,2020-01-01 01:00,-5.9,0.0,1.7,2.0,40.0,1.6,-17.3,1021.8,1033.0,0.0,0.0,0.0,2000.0,-2.4,-2.9,0.0,2.0,15.0,22.0,1.1,-21.7,1021.6,1030.6,0.0,0.0,0.0,2000.0,-1.6,265
2,2020-01-01 02:00,-5.7,0.0,0.1,0.0,42.0,1.7,-16.5,1021.6,1032.8,0.0,0.0,0.0,1988.0,-2.4,-3.1,0.0,1.9,6.0,23.0,1.1,-21.3,1021.6,1030.6,0.0,0.0,0.0,2000.0,-1.9,291
3,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46.0,1.8,-15.4,1021.5,1032.7,0.0,0.0,0.0,2000.0,-2.7,-2.8,0.0,4.5,1.0,24.0,1.2,-20.6,1021.1,1030.1,0.0,0.0,0.0,2000.0,-2.0,284
4,2020-01-01 04:00,-5.4,0.0,0.0,0.0,50.0,2.0,-14.2,1021.6,1032.8,0.0,0.0,0.0,1908.0,-2.5,-3.1,0.0,1.9,5.0,35.0,1.7,-16.4,1021.0,1030.0,0.0,0.0,0.0,2000.0,-2.1,363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2021-12-31 19:00,-6.4,0.0,2.2,11.0,34.0,1.3,-19.7,1021.1,1032.3,0.0,0.0,0.0,2000.0,-4.5,0.5,0.0,2.0,14.0,22.0,1.4,-18.8,1019.1,1028.0,0.0,0.0,0.0,5000.0,-1.6,309
17540,2021-12-31 20:00,-6.3,0.0,4.1,14.0,35.0,1.3,-19.2,1021.2,1032.4,0.0,0.0,0.0,2000.0,-5.3,0.0,0.0,2.2,14.0,22.0,1.3,-19.2,1019.1,1028.0,0.0,0.0,0.0,5000.0,-2.1,320
17541,2021-12-31 21:00,-6.7,0.0,4.8,14.0,36.0,1.3,-19.3,1021.2,1032.4,0.0,0.0,0.0,2000.0,-5.7,-0.4,0.0,4.1,13.0,23.0,1.4,-19.0,1018.7,1027.6,0.0,0.0,0.0,5000.0,-2.5,323
17542,2021-12-31 22:00,-7.5,0.0,3.0,14.0,37.0,1.3,-19.7,1021.8,1033.1,0.0,0.0,0.0,2000.0,-6.2,-0.8,0.0,3.2,11.0,23.0,1.3,-19.4,1019.5,1028.4,0.0,0.0,0.0,5000.0,-2.2,355


In [118]:
train_data['일시'] = pd.to_datetime(train_data['일시'], format='%Y-%m-%d %H:%M', errors='raise')
test_data['일시'] = pd.to_datetime(test_data['일시'], format='%Y-%m-%d %H:%M', errors='raise')
final_data['일시'] = pd.to_datetime(final_data['일시'], format='%Y-%m-%d %H:%M', errors='raise')

2. 요일 추가

In [28]:
train_data['weekday'] = train_data['일시'].dt.day_name()
train_data['weekday'] = train_data['weekday'].str.slice(start=0, stop=3)
test_data['weekday'] = test_data['일시'].dt.day_name()
test_data['weekday'] = test_data['weekday'].str.slice(start=0, stop=3)

2. 공휴일 추가

In [116]:
train_data['holiday'] = 0
test_data['holiday'] = 0
final_data['holiday'] = 1

In [19]:
month_2020 = [1, 1, 1, 1, 3, 4, 5, 6, 8, 8, 9, 10, 10, 10, 10, 12]
day_2020 = [1, 24, 25, 26, 1, 30, 5, 6, 15, 17, 30, 1, 2, 3, 9, 25]

for i in range(len(month_2020)):
    m = month_2020[i]
    d = day_2020[i]
    
    train_data.loc[(train_data['일시'].dt.year==2020) & (train_data['일시'].dt.month==m) & (train_data['일시'].dt.day==d), 'holiday'] = 1

In [20]:
month_2021 = [1, 2, 2, 2, 3, 5, 5, 6, 8, 9, 9, 9, 10, 10, 12]
day_2021 = [1, 11, 12, 13, 1, 5, 19, 6, 15, 20, 21, 22, 3, 9, 25]

for i in range(len(month_2021)):
    m = month_2021[i]
    d = day_2021[i]
    
    train_data.loc[(train_data['일시'].dt.year==2021) & (train_data['일시'].dt.month==m) & (train_data['일시'].dt.day==d), 'holiday'] = 1

In [30]:
month_2022 = [1, 1, 2, 2, 3, 5, 5, 6, 8]
day_2022 = [1, 31, 1, 2, 1, 5, 8, 6, 15]

for i in range(len(month_2022)):
    m = month_2022[i]
    d = day_2022[i]
    
    test_data.loc[(test_data['일시'].dt.year==2022) & (test_data['일시'].dt.month==m) & (test_data['일시'].dt.day==d), 'holiday'] = 1

3. 명절 추가

In [117]:
train_data['Korean_holiday'] = 0
test_data['Korean_holiday'] = 0
final_data['Korean_holiday'] = 1

In [32]:
year = [2020, 2020, 2020, 2020, 2020, 2020,
        2021, 2021, 2021, 2021, 2021, 2021]

month = [1, 1, 1, 9, 10, 10,
         2, 2, 2, 9, 9, 9]
month2 = [1, 2, 2]

day = [24, 25, 26, 30, 1, 2,
       11, 12, 13, 20, 21, 22]
day2 = [31, 1, 2]

for i in range(len(year)):
    y = year[i]
    m = month[i]
    d = day[i]
    
    train_data.loc[(train_data['일시'].dt.year==y) & (train_data['일시'].dt.month==m) & (train_data['일시'].dt.day==d), 'Korean_holiday'] = 1
    
for i in range(3):
    m = month2[i]
    d = day2[i]
    
    test_data.loc[(test_data['일시'].dt.year==2022) & (test_data['일시'].dt.month==m) & (test_data['일시'].dt.day==d), 'Korean_holiday'] = 1

In [33]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

4. year, month, time 추가

In [124]:
train_data['year'] = train_data['일시'].dt.year
train_data['month'] = train_data['일시'].dt.month
train_data['time'] = train_data['일시'].dt.hour

test_data['year'] = test_data['일시'].dt.year
test_data['month'] = test_data['일시'].dt.month
test_data['time'] = test_data['일시'].dt.hour

final_data['year'] = 2022
final_data['month'] = 9
final_data['time'] = final_data['일시'].dt.hour

5. 주말 추가

In [1]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

In [5]:
train_data['weekend'] = 0
test_data['weekend'] = 0

train_data.loc[(train_data['weekday']=='Sat') | (train_data['weekday']=='Sun'), 'weekend']=1
test_data.loc[(test_data['weekday']=='Sat') | (test_data['weekday']=='Sun'), 'weekend']=1

train_data.drop('weekday', axis=1, inplace=True)
test_data.drop('weekday', axis=1, inplace=True)

In [120]:
final_data['weekend'] = 0

In [6]:
test_data.shape

(5976, 36)

In [9]:
train_data.head()

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),...,부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간,holiday,Korean_holiday,year,month,time,weekend
0,2020-01-01 00:00:00,-6.5,0.0,0.0,0.0,38.0,1.4,-18.5,1022.6,1033.8,...,0.0,2000.0,-1.4,265,1,0,2020,1,0,0
1,2020-01-01 01:00:00,-5.9,0.0,1.7,2.0,40.0,1.6,-17.3,1021.8,1033.0,...,0.0,2000.0,-1.6,265,1,0,2020,1,1,0
2,2020-01-01 02:00:00,-5.7,0.0,0.1,0.0,42.0,1.7,-16.5,1021.6,1032.8,...,0.0,2000.0,-1.9,291,1,0,2020,1,2,0
3,2020-01-01 03:00:00,-5.6,0.0,0.0,0.0,46.0,1.8,-15.4,1021.5,1032.7,...,0.0,2000.0,-2.0,284,1,0,2020,1,3,0
4,2020-01-01 04:00:00,-5.4,0.0,0.0,0.0,50.0,2.0,-14.2,1021.6,1032.8,...,0.0,2000.0,-2.1,363,1,0,2020,1,4,0


In [8]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [128]:
final_data.to_csv('final_data.csv', index=False)